Many physical problems require the evaluation of all pairwise interactions of a large number of particles, so-called N-body problems. These problems arise in molecular dynamics, astrodynamics and electromagnetics among others. 

Their pairwise interactions can be expressed as:

\begin{equation}
f_i = \sum_{j=1}^n{P \left(\boldsymbol{x}_i, \boldsymbol{x}_j \right)w_j} \ \ \ \text{for } i=1,2,...,n 
\end{equation}

*  where subscripts $i$,  $j$ respectively denote *target* and *source*
*  $f_i$ can be a *potential* (or *force*) at target point $i$
*  $w_j$ is the *source weight* 
*  $\boldsymbol{x}_i, \boldsymbol{x}_j$ are the *spatial positions* of particles 
*  $P \left(\boldsymbol{x}_i, \boldsymbol{x}_j \right)$ is the *interaction kernel*. 

In order to evalute the potential $f_i$ at a target point $i$, we have to loop over each source particle $j$. Since there are $n$ target points $i$, this 'brute-force' approach costs $\mathcal{O} \left(n^2 \right)$ operations. 

One possible approach in this kind of problem is to define a few classes, say `Point` and `Particle` and then loop over the objects and perform the necessary point-to-point calculations.  

In [ ]:
import numpy

In [ ]:
class Point():
    """    
    Arguments:
        domain: the domain of random generated coordinates x,y,z, 
                default=1.0
    
    Attributes:
        x, y, z: coordinates of the point
    """
    def __init__(self, domain=1.0):
        self.x = domain * numpy.random.random()
        self.y = domain * numpy.random.random()
        self.z = domain * numpy.random.random()
            
    def distance(self, other):
        return ((self.x-other.x)**2 + 
                (self.y-other.y)**2 + 
                (self.z-other.z)**2)**.5

In [ ]:
class Particle(Point):
    """    
    Attributes:
        m: mass of the particle
        phi: the gravitational potential of the particle
    """
    
    def __init__(self, domain=1.0, m=1.0):
        Point.__init__(self, domain)
        self.m = m
        self.phi = 0.

Now we create a list of `n` random particles, define a function to calculate their interaction via direct summation and run!

In [ ]:
n = 1000
particles = [Particle(m= 1 / n) for i in range(n)]

In [ ]:
def direct_sum(particles):
    """
    Calculate the gravitational potential at each particle
    using direct summation method.

    Arguments:
        particles: the list of particles

    """
    for i, target in enumerate(particles):
        for source in (particles[:i] + particles[i+1:]):
            r = target.distance(source)
            target.phi += source.m / r

In [ ]:
direct_sum(particles)

There was a noticeable lag there.  How long does this thing take for 1000 particles?

In [ ]:
%timeit direct_sum(particles)

## How do we use Numba on this problem?

Problem: Numba doesn't support jitting native Python classes.  There is a `jit_class` structure in Numba but it's still in early development.

But it's nice to have attributes for literate programming.

Solution: NumPy custom dtypes.

In [ ]:
particle_dtype = numpy.dtype({'names':['x','y','z','m','phi'], 
                             'formats':[numpy.double, 
                                        numpy.double, 
                                        numpy.double, 
                                        numpy.double, 
                                        numpy.double]})

In [ ]:
myarray = numpy.ones(3, dtype=particle_dtype)

In [ ]:
myarray

You can access an individual "attribute" like this:

In [ ]:
myarray[0]['x']

In [ ]:
from numba import jit

## Exercise

Write a function that takes the arguments `n` (number of particles), `m` (mass of every particle) and a domain within to generate a random number (as in the class above).
It should create an array with `n` elements and `dtype=particle_dtype` and then return that array.

In [ ]:
@jit(nopython=True)
def create_n_random_particles(n, m, domain = 1):
    '''
    Creates `n` particles with mass `m` with random coordinates
    between 0 and `domain`
    '''
    parts = numpy.zeros((n), dtype=particle_dtype)
    #attribute access only in @jitted function
    for i in parts:
        i.x = numpy.random.random() * domain 
        i.y = numpy.random.random() * domain
        i.z = numpy.random.random() * domain
        i.m = m
        i.phi = 0
    return parts

**Note**: You can use "attribute" access on dtypes but there's a caveat.  If you need to debug this function without the decorator, you have to change them back to array access form.  

In [ ]:
%timeit create_n_random_particles(1000, 1/1000)

In [ ]:
parts = create_n_random_particles(1000, .001)

We don't have a `distance` method anymore, so we need to write a function to take care of that.

## Exercise
Write a JITted function `distance` to calculate the distance between two particles of dtype `particle_dtype`

In [ ]:
@jit(nopython=True)
def distance(part1, part2):
    '''calculate the distance between two particles'''
    return ((part1.x - part2.x)**2 + 
            (part1.y - part2.y)**2 + 
            (part1.z - part2.z)**2)**.5

In [ ]:
%%timeit
distance(parts[0], parts[1])

## Exercise
Modify the `direct_sum` function above to instead work a NumPy array of particles.  Loop over each element in the array and calculate its total potential.

In [ ]:
@jit(nopython=True)
def direct_sum(particles):
    for i, target in enumerate(particles):
        for j, source in enumerate(particles):
            if i != j:
                r = distance(target, source)
                target.phi += source.m / r
                
    return particles

In [ ]:
%timeit direct_sum(parts)